<a href="https://colab.research.google.com/github/charge0315/colab-musicgen-finetuning/blob/main/colab-musicgen-finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MusicGen-Large ファインチューニング & 生成ノートブック

このノートブックでは、Facebook (Meta) の `audiocraft` ライブラリを使用して、MusicGen-Largeモデルを独自のデータセット（約17,000曲）でファインチューニングし、その後、学習したモデルを使って楽曲生成を行います。

**前提条件:**
1.  Google Colabのランタイムタイプを **GPU** に設定してください。（MusicGen-Largeはメモリを大量に消費するため、**A100 GPU** の使用を強く推奨します。V100でも設定次第では動作しますが、T4ではメモリ不足になる可能性が高いです。）
2.  Google Driveに以下のフォルダ構成でデータを配置してください。
    *   `My Drive/MusicGen_Dataset/`
        *   `wav_chunks/` (wavファイルを格納)
        *   `metadata.jsonl` (提供された形式のJSONLファイル)

### 1. 環境構築 (Environment Setup)
必要なライブラリ (`audiocraft`, `xformers` 等) をインストールし、リポジトリをクローンします。

In [3]:
# @title 環境構築とライブラリのインストール
import os

# 再実行時のトラブル防止のためホームディレクトリに戻る
%cd /content

# システム依存関係のインストール
!sudo apt-get update
!sudo apt-get install -y ffmpeg libavcodec-dev libavdevice-dev libavfilter-dev libavformat-dev libavutil-dev pkg-config

# 既存のフォルダがある場合は削除してクリーンにする
!rm -rf audiocraft

# 必要なPythonライブラリのアップグレード
!pip install -U pip setuptools wheel

# PyTorch, TorchAudio, xformersのインストール
# Colabの環境に合わせて最新(に近い)安定版を入れます
!pip install -U torch torchaudio xformers --index-url https://download.pytorch.org/whl/cu121

# WandBのインストール
!pip install -U wandb

# ソースコードから最新機能を使うためにclone
!git clone https://github.com/facebookresearch/audiocraft.git
%cd audiocraft

# requirements.txt から torch, torchaudio, torchvision の依存関係行を削除
# これにより、audiocraftが指定する古いバージョン制約を無視し、
# 上記でインストールした最新の互換バージョンを使用させます。
!sed -i '/^torch/d' requirements.txt

!pip install -e .

print("インストールが完了しました。")

/content
Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:2 https://cli.github.com/packages stable InRelease [3,917 B]
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 3,917 B in 1s (2,837 B/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to pro

### (オプション) 素のMusicGenモデルでの生成テスト
ファインチューニングを行う前に、オリジナルの `facebook/musicgen-large` モデルでどのような曲が生成されるか確認できます。
※このセルを実行するとモデルがメモリにロードされます。ファインチューニング実行前にメモリ不足になる可能性があるため、確認後は「ランタイムの再起動」を行うことを推奨します。

In [ ]:
# @title 素のMusicGen-Largeで生成テスト
from audiocraft.models import MusicGen
from audiocraft.data.audio import audio_write
from IPython.display import Audio, display
import torch

# モデルのロード (facebook/musicgen-large)
print("素のモデルをロード中...")
model = MusicGen.get_pretrained('facebook/musicgen-large')

# 生成パラメータ設定
model.set_generation_params(
    use_sampling=True,
    top_k=250,
    duration=10  # テスト用に10秒
)

# プロンプト (自由に書き換えてください)
descriptions = [
    "A dynamic heavy metal song with fast drums and guitar solo",
    "Relaxing jazz piano with soft rain background",
]

print("生成中...")
wav = model.generate(descriptions)

# 再生
for idx, one_wav in enumerate(wav):
    filename = f"base_generated_{idx}"
    audio_write(filename, one_wav.cpu(), model.sample_rate, strategy="loudness", loudness_compressor=True)
    print(f"\nPrompt: {descriptions[idx]}")
    display(Audio(filename + ".wav"))

# メモリ解放 (念のため)
del model
torch.cuda.empty_cache()

### 2. Google Driveのマウント (Mount Google Drive)
データセットの読み込みと、学習済みモデルの保存先としてGoogle Driveをマウントします。

In [ ]:
# @title Google DriveのマウントとWandBログイン
from google.colab import drive
from google.colab import userdata
import wandb
import os

drive.mount('/content/drive')

# WandBログイン
try:
    wandb_api_key = userdata.get('WANDB_API_KEY')
    wandb.login(key=wandb_api_key)
    # 環境変数にもセットしておく（念のため）
    os.environ["WANDB_API_KEY"] = wandb_api_key
    print("WandBにログインしました。")
except Exception as e:
    print(f"WandBログインエラー/スキップ: {e}")
    print("シークレット 'WANDB_API_KEY' が設定されているか確認してください。")

# 作業用ディレクトリの設定（必要に応じて変更してください）
dataset_root = '/content/drive/MyDrive/MusicGen_Dataset'
output_dir = '/content/drive/MyDrive/MusicGen_Checkpoints'

os.makedirs(output_dir, exist_ok=True)
print(f"データセットパス: {dataset_root}")
print(f"モデル保存先: {output_dir}")

### 3. データセットの準備 (Dataset Preparation)
提供されたJSONLファイルを読み込み、Audiocraftが学習に使用できる形式に変換します。
ここでは、学習用(train)と検証用(valid)にデータを分割し、パスを絶対パスに書き換えます。

In [ ]:
# @title データセットのマニフェスト作成
import json
import random
from pathlib import Path

# 設定
input_jsonl = Path(dataset_root) / 'metadata.jsonl'
train_jsonl = Path(dataset_root) / 'train.jsonl'
valid_jsonl = Path(dataset_root) / 'valid.jsonl'
validation_split = 0.05  # 5%を検証用に回す

# データの読み込みと変換
data = []
if not input_jsonl.exists():
    print(f"エラー: {input_jsonl} が見つかりません。パスを確認してください。")
else:
    with open(input_jsonl, 'r', encoding='utf-8') as f:
        for line in f:
            item = json.loads(line)
            # パスを絶対パスに変換
            # item['path'] が "audio/track1.wav" のような相対パスであることを想定
            abs_path = Path(dataset_root) / item['path']

            if abs_path.exists():
                # Audiocraftに必要なフィールドを整理
                entry = {
                    "path": str(abs_path),
                    "duration": item.get("duration", 30.0),
                    "sample_rate": item.get("sample_rate", 32000),
                    "amplitude": item.get("amplitude", None),
                    "description": item.get("description", ""),
                }
                data.append(entry)
            else:
                print(f"警告: 音声ファイルが見つかりません - {abs_path}")

    # シャッフルして分割
    random.shuffle(data)
    split_idx = int(len(data) * (1 - validation_split))
    train_data = data[:split_idx]
    valid_data = data[split_idx:]

    # 保存
    def save_jsonl(data_list, output_path):
        with open(output_path, 'w', encoding='utf-8') as f:
            for entry in data_list:
                f.write(json.dumps(entry) + '\n')

    save_jsonl(train_data, train_jsonl)
    save_jsonl(valid_data, valid_jsonl)

    print(f"データセット準備完了:")
    print(f"  Train: {len(train_data)} 曲 -> {train_jsonl}")
    print(f"  Valid: {len(valid_data)} 曲 -> {valid_jsonl}")

### 4. 学習設定ファイルの作成 (Create Configuration)
MusicGen-Largeのファインチューニング設定ファイル (`finetune.yaml`) を作成します。
GPUメモリに合わせて `batch_size` を調整してください。

In [ ]:
# @title 設定ファイル (finetune.yaml) の作成
import yaml

config = {
    "continue_from": "//pretrained/facebook/musicgen-large",
    "defaults": [
        {"dset": "audio/default"},
        {"solver": "musicgen/musicgen_base_32khz"},
        "_self_"
    ],
    "dset": {
        "train": {"paths": [str(train_jsonl)]},
        "valid": {"paths": [str(valid_jsonl)]},
        "evaluate": {"paths": [str(valid_jsonl)]},
        "processor": {"sample_rate": 32000, "channels": 1} # モノラル想定
    },
    "solver": {
        "max_epochs": 10,  # 必要に応じて増減
        "max_gen_epochs": 2, # 生成テストを行う頻度
        "batch_size": 4,   # A100なら4-8, V100なら2-4, T4なら1 (OOMの場合は減らす)
        "gradient_accumulation_steps": 4, # バッチサイズが小さい場合は増やす
        "audio_box": {
            "sample_rate": 32000,
            "segment_duration": 30.0
        }
    },
    "checkpoint": {
        "save_last": True,
        "save_every": 1,
        "keep_last": 5,
        "save_folder": output_dir # Driveに保存
    },
    "logging": {
        "log_tensorboard": True,
        "log_wandb": True,
        "wandb": {
            "project": "musicgen-finetune"
        }
    },
    # Largeモデル用の最適化設定 (FSDPなど)
    "fsdp": {
        "use": True
    }
}

# YAMLとして保存
config_path = '/content/audiocraft/config/solver/musicgen/musicgen_finetune.yaml'
with open(config_path, 'w') as f:
    yaml.dump(config, f)

print(f"設定ファイルを保存しました: {config_path}")

### 5. 学習の実行 (Run Training)
`dora` コマンドを使用して学習を開始します。
※学習には数時間〜数日かかる場合があります。

In [ ]:
# @title 学習の開始
# dora run コマンドで学習を実行
# solverに先ほど作成した設定を指定します
!dora run solver=musicgen/musicgen_finetune

### 6. 生成・推論 (Inference / Generation)
学習したモデル（チェックポイント）をロードして、テキストから楽曲を生成します。

In [ ]:
# @title 学習済みモデルでの楽曲生成
from audiocraft.models import MusicGen
from audiocraft.data.audio import audio_write
import torch

# 最新のチェックポイントを自動的に探す、またはパスを直接指定
# 例: model_path = "/content/drive/MyDrive/MusicGen_Checkpoints/checkpoint.th"
# ここでは学習直後と仮定して、保存先ディレクトリを指定してロードする方法を示します
# 注意: 実際のチェックポイントファイルパスを確認して指定してください

# 学習が完了していない場合やテスト用に、ここではベースモデルをロードする例を書きますが、
# ファインチューニング後は以下のようにパスを指定してロードします：
# model = MusicGen.get_pretrained(output_dir) # 保存フォルダを指定

print("モデルをロード中...")
# デモ用にfacebookのモデルをロードする場合:
# model = MusicGen.get_pretrained('facebook/musicgen-large')

# ファインチューニングしたモデルをロードする場合 (パスは適宜修正):
# doraのログに出力されたチェックポイントパス(.thファイル)を指定するのが確実です
# model = MusicGen.get_pretrained('/content/drive/MyDrive/MusicGen_Checkpoints/<sig>/checkpoint.th')

# --- 以下、生成コード ---
# モデルのパラメータ設定
model.set_generation_params(
    use_sampling=True,
    top_k=250,
    duration=30  # 生成する秒数
)

# プロンプトの設定
descriptions = [
    "A fast paced rock song with heavy guitar riffs",
    "Lo-fi hip hop beat with a chill vibe",
    "Orchestral symphony with dramatic strings"
]

print("生成を開始します...")
wav = model.generate(descriptions)  # 生成実行

# 音声の保存と再生
for idx, one_wav in enumerate(wav):
    # 音声を保存
    filename = f"generated_{idx}"
    audio_write(filename, one_wav.cpu(), model.sample_rate, strategy="loudness", loudness_compressor=True)
    print(f"保存しました: {filename}.wav")

    # Colab上で再生するためのウィジェット表示
    from IPython.display import Audio, display
    display(Audio(filename + ".wav"))